# Attachment Protocol - Bob
## Role: Receiver

## This notebook should be run alongside [Alice](http://localhost:8888/lab/tree/attachment.ipynb), who will send Bob two attachments.

In this notebook Bob instatiates his controller and sets up a handler for attachment protocol messages, all files he receives he will save in the [received_files folder](http://localhost:8889/tree/received_files). It should currently be empty.


###  Intialise Controller

In [ ]:
%autoawait
import time
import asyncio
import base64
from attachment_controller.attachment_controller import AttachmentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

# Based on the aca-py agent you wish to control
attach_controller = AttachmentController(admin_url=ADMIN_URL, webhook_host=WEBHOOK_HOST, webhook_base=WEBHOOK_BASE, webhook_port=WEBHOOK_PORT)

###  Check if active connection exists

In [ ]:
response = await attach_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")


### Instantiate Listener

The "attachment" event is emitted using PyPubSub when the controller receives a atatchment webhook from the agent. This happens everytime the agent receives an attachment.

In this handler we receive the attachment payload, decode it and save it in the received files folder using the filename provided in the protocol. 

Currently, this has been tested for basic text files and for images. Feel free to try additional file types.

In [ ]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)
    print("Attachment Received")
    data = payload['content']
    filename = payload['filename']
    file_content = base64.b64decode(data)
#     fh = open("image_received.png", "wb")
#     fh.write(text)
#     fh.close()
    filepath = f"{filename}"
    f = open(filepath,'wb')
    f.write(file_content)
    f.close()
attach_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(attach_controller.listen_webhooks())
attach_controller.register_listeners([attach_listener], defaults=True)


# End of Tutorial

Be sure to terminate the controller so you can run another tutorial.


In [ ]:
response = await attach_controller.terminate()
print(response)